In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import itertools
import csv
from tkinter import ttk, StringVar
import tkinter as tk
from tkinter import messagebox
import sys

"""
Function definitions
"""
def volumeunit(vol):
    """ Determines the units and conversion factors for different volume 
        units """
    if vol == 'bbl':
        divisor = 1000
        vunit = 'Mbbl'
    if vol == 'MMscf':
        divisor = 1000
        vunit = 'Bscf'
    if vol == 'Mscf':
        divisor = 1000000
        vunit = 'Bscf'
    return divisor,vunit

def DCA(qi,qt,di,b):
    """ Calculates the hyperbolic decline curve """
    t = np.array([0],dtype=int);
    q = np.array([qi],dtype=float);
    for i in range(1,1000):
        # modification to flow
        newflow = q[0] / np.power(1 + b * di * i, 1 / b)
        t = np.append(t,i)
        q = np.append(q,newflow)
        
        # breaks calculation when flow is below the minimum desired
        if newflow <= qt:
            break;      
    return t,q

def totalprod(b,D,qi,qt):
    """ Calculates the production for each condition """
    Np = np.power(qi,b)/(D*(1-b))*(1/np.power(qi,b-1)-1/np.power(qt,b-1))
    return Np

def baseDCA(Qi,Qt,Di,b,vol):
    """ Calculate the decline curve, plot it and save figure """
    divisor,vunit = volumeunit(vol)
    T0,Q0 = DCA(Qi,Qt,Di,b)
    
    dca = plt.figure(1,figsize=(8,6), dpi=100)
    plt.plot(T0,Q0,'bo-')
    plt.yscale('log')
    plt.ylim(np.power(10,np.floor(np.log10(Q0[-1]))),np.power(10,np.ceil(np.log10(Q0[0]))))
    plt.xlim(0,np.around(np.max(T0),-1))
    plt.xlabel('Month',color='k',fontsize=12)
    plt.ylabel('HC production ('+vol+'/month)',color='k',fontsize=12)
    dca.savefig('DCA.png',format='png',dpi=600)
    plt.gcf().clear()
    
    with open('ProdHist.csv', 'w', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['Month','HC production ('+vunit+')'])
        writer.writerows(zip(T0,Q0))     
         
def iterativetool(QiMax,QiMin,Qinum,QtMax,QtMin,Qtnum,DiMax,DiMin,Dinum,\
                  bMax,bMin,bnum,vol,NpDCA,T0,Q0):
    """ Takes the matrix input from the user, calculates the extremes, plots 
        the curves, and outputs the results in tabular format """
    Qiloop = np.linspace(QiMax,QiMin,Qinum)
    Qtloop = np.linspace(QtMax,QtMin,Qtnum)
    Diloop = np.linspace(DiMax,DiMin,Dinum)
    bloop = np.linspace(bMax,bMin,bnum)
    divisor,vunit = volumeunit(vol)
    masterlist = [list(Qiloop),list(Qtloop),list(Diloop),list(bloop)]
    listing = list(itertools.product(*masterlist))
    totalestimates = len(listing)
    Np = np.array([])
    for i in range(totalestimates):
        qi,qt,di,b = listing[i]
        temp = totalprod(b,di,qi,qt)/divisor
        Np = np.append(Np,temp)
    
    Np_sorted = np.msort(Np)    
    Maxindex = np.where(Np==max(Np))
    Minindex = np.where(Np==min(Np))

    # Extremes - create and save figure
    maxQi,maxQt,maxDi,maxb = listing[int(Maxindex[0])]
    TM,QM = DCA(maxQi,maxQt,maxDi,maxb)
    minQi,minQt,minDi,minb = listing[int(Minindex[0])]
    Tm,Qm = DCA(minQi,minQt,minDi,minb)
    
    # Figure generation and save
    dca2 = plt.figure(2,figsize=(8,6), dpi=100)
    plt.hold(True)
    plt.plot(TM,QM,'rs-',label='Maximum',markersize=4,markevery=5)
    plt.plot(T0,Q0,'b^-',label='DCA',markersize=4,markevery=3)
    plt.plot(Tm,Qm,'ko-',label='Minimum',markersize=4,markevery=1)
    plt.yscale('log')
    plt.ylim(np.power(10,np.floor(np.log10(QM[-1]))),np.power(10,np.ceil(np.log10(Q0[0]))))
    plt.xlim(0,np.size(TM))
    plt.xlabel('Month',color='k',fontsize=12)
    plt.ylabel('HC production ('+vol+'/month)',color='k',fontsize=12)
    plt.legend()
    dca2.savefig('DCA2.png',format='png',dpi=600)
    plt.gcf().clear()
    
    d = [Tm,Qm,T0,Q0,TM,QM]

    probabilitytableandfigure(totalestimates,Np_sorted,vunit,NpDCA)
    with open('MaxMinDCA.csv', 'w', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['Maximum','','DCA','','Minimum',''])
        writer.writerow(['Month','HC production ('+vunit+')','Month','HC production ('+vunit+')','Month','HC production ('+vunit+')'])
        for values in itertools.zip_longest(*d):
            writer.writerow(values)        

def probabilitytableandfigure(totalestimates,Np_sorted,vunit,NpDCA):
    """ Generate, plot, and write probability table """

    prob = np.linspace(1,0,totalestimates)
    probfig = plt.figure(3,figsize=(8,6), dpi=100)
    plt.plot(Np_sorted,prob,'k',label='Reserve',lw=2)
    plt.xlim(0,np.around(Np_sorted[-1]))
    plt.ylim(0,1)
    p90,p50,p10 = np.interp([0.1,0.5,0.9],1-prob,Np_sorted)
    plt.annotate('P10',xy=(p10,0.10),xytext=(p10*1.1,0.25),color='blue',\
                 arrowprops=dict(facecolor='blue', shrink=0.05))
    plt.annotate('P50',xy=(p50,0.50),xytext=(p50*1.3,0.55),color='red',\
                 arrowprops=dict(facecolor='red', shrink=0.05))
    plt.annotate('P90',xy=(p90,0.90),xytext=(p90*3,0.95),color='green',\
                 arrowprops=dict(facecolor='green', shrink=0.05))
    plt.xlabel('Reserves ('+vunit+')',color='k',fontsize=12)
    plt.ylabel('Probability',color='k',fontsize=12)
    col_labels=['Reserves ('+vunit+')']
    row_labels=['DCA','P10','P50','P90']
    table_vals=[["%8.1f" % NpDCA],["%8.1f" % p10],["%8.1f" % p50],["%8.1f" % p90]]
    plt.table(cellText=table_vals,
                      colWidths = [0.3],
                      rowLabels=row_labels,
                      colLabels=col_labels,
                      loc='center right')
    probfig.savefig('Probability.png',format='png',dpi=600)
    plt.gcf().clear()

    with open('Probability.csv', 'w', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['Reserve ('+vunit+')','Probability'])
        writer.writerows(zip(Np_sorted, prob))     
    output(vunit,NpDCA,p10,p50,p90)
    
def output(vunit,NpDCA,p10,p50,p90):
    print('Reserve estimates ('+vunit+')')
    print('DCA = %8.2f (%s)' % (NpDCA,vunit))
    print('P10 = %8.2f (%s)' % (p10,vunit))
    print('P50 = %8.2f (%s)' % (p50,vunit))
    print('P90 = %8.2f (%s)' % (p90,vunit))

"""
Button actions
""" 
def declinecurveanalysis():
   vol = selected.get()
   
   while True:
       try:
           QiDCA = float(Qi.get())
           QtDCA = float(Qt.get())
           DiDCA = float(Di.get())
           bDCA = float(b.get())
       except ValueError:
           messagebox.askretrycancel(title="Error",\
                                     message="The input has to be numerical")
           window.mainloop()
       else:
           break
       
   if QiDCA < QtDCA:
       messagebox.askretrycancel(title="Error",\
                                 message="Qt cannot be larger than Qi!")
       window.mainloop()
   elif bDCA >= 1 or bDCA < 0 or DiDCA >=1 or DiDCA < 0:
       messagebox.askretrycancel(title="Error",\
                                 message="0 < b < 1 & 0 < Di 1")
       window.mainloop()

   baseDCA(QiDCA,QtDCA,DiDCA,bDCA,vol)   
   
   
def RUN():
   vol = selected.get()
   numDi = int(comboDi.get())
   numQi = int(comboQi.get())
   numQt = int(comboQt.get())
   numb = int(combob.get())
   
   while True:
       try:
           QiDCA= float(Qi.get())
           QtDCA = float(Qt.get())
           DiDCA = float(Di.get())
           bDCA = float(b.get())
   
           MaxQi = float(QiM.get())
           MaxQt = float(QtM.get())
           MaxDi = float(DiM.get())
           Maxb = float(bM.get())
   
           MinQi = float(Qim.get())
           MinQt = float(Qtm.get())
           MinDi = float(Dim.get())
           Minb = float(bm.get())
       except ValueError:
           messagebox.askretrycancel(title="Error",\
                                     message="The input has to be numerical")
           window.mainloop()
       else:
           break
   
   if MaxQi<=MinQi or MaxQt<=MinQt or MaxDi<=MinDi or Maxb<=Minb:
       messagebox.askretrycancel(title="Error",message=\
                                 "Minimum value should be smaller than Maximum")
       window.mainloop()
   elif Maxb >= 1 or Minb < 0 or MaxDi >= 1 or MinDi < 0 or bDCA >= 1 or \
   bDCA < 0 or DiDCA >=1 or DiDCA < 0:
       messagebox.askretrycancel(title="Error",message="0 < b < 1")
       window.mainloop()
   elif QiDCA < QtDCA or min(MaxQi,MinQi) < max(MaxQt,MinQt):
       messagebox.askretrycancel(title="Error",\
                                 message="Qt cannot be larger than Qi!")
       window.mainloop()
   
   divisor,vunit = volumeunit(vol)
   NpDCA = totalprod(bDCA,DiDCA,QiDCA,QtDCA)/divisor
   T0,Q0 = DCA(QiDCA,QtDCA,DiDCA,bDCA)
   iterativetool(MaxQi,MinQi,numQi,MaxQt,MinQt,numQt,MaxDi,MinDi,numDi,\
                  Maxb,Minb,numb,vol,NpDCA,T0,Q0)


def on_closing():
   if messagebox.askokcancel("Quit", "Do you want to quit?"):
       window.destroy()
       sys.exit()
"""
GUI setup
"""

window = tk.Tk()
window.title("Economics worksheet")
window.geometry('550x450')

# Decline Curve Analysis input
lbl = ttk.Label(window,font=("Arial Bold", 12),foreground="red",\
            text="1. Fill in the hyperbolic decline curve fitting parameters")
lbl.grid(column=3, row=0, columnspan=3,sticky='W',pady=10)

Di = ttk.Entry(window)
Qi = ttk.Entry(window)
Qt = ttk.Entry(window)
b = ttk.Entry(window)

Di.grid(row=2, column=4)
Qi.grid(row=3, column=4)
Qt.grid(row=4, column=4)
b.grid(row=5, column=4)

lblDi = ttk.Label(window, text="Di")
lblDi.grid(column=3, row=2)

lblQi = ttk.Label(window, text="Qi")
lblQi.grid(column=3, row=3)

lblQt = ttk.Label(window, text="Qt")
lblQt.grid(column=3, row=4)

lblb = ttk.Label(window, text="b")
lblb.grid(column=3, row=5)

lblDiunit = ttk.Label(window, text="1/month")
lblDiunit.grid(column=5, row=2, sticky='w')

lblQiunit = ttk.Label(window, text="vol/month")
lblQiunit.grid(column=5, row=3, sticky='w')

lblQtunit = ttk.Label(window, text="vol/month")
lblQtunit.grid(column=5, row=4, sticky='w')

# Volume unit selection
lbl = ttk.Label(window,font=("Arial Bold", 12),foreground="red",\
            text="2. Select the volume unit")
lbl.grid(column=3, row=6, columnspan=3,sticky='W',pady=10)

selected = StringVar(value="bbl")
rad1 = ttk.Radiobutton(window,text='bbl', value="bbl", variable=selected)
rad2 = ttk.Radiobutton(window,text='Mscf', value="Mscf", variable=selected)
rad3 = ttk.Radiobutton(window,text='MMscf', value="MMscf", variable=selected)

rad1.grid(column=3, row=8)
rad2.grid(column=4, row=8)
rad3.grid(column=5, row=8)

# Data spread input
lbl = ttk.Label(window,font=("Arial Bold", 12),foreground="red",\
            text="3. Fill in the parameter ranges." )
lbl.grid(column=3, row=9, columnspan=3,sticky='W',pady=10)

lblm = ttk.Label(window, text="Min")
lblm.grid(column=3, row=12)

lblM = ttk.Label(window, text="Max")
lblM.grid(column=4, row=12)

lbldiv = ttk.Label(window, text="Division")
lbldiv.grid(column=5, row=12)

lblDi = ttk.Label(window, text="Di")
lblDi.grid(column=2, row=13)

lblQi = ttk.Label(window, text="Qi")
lblQi.grid(column=2, row=14)

lblQt = ttk.Label(window, text="Qt")
lblQt.grid(column=2, row=15)

lblb = ttk.Label(window, text="b")
lblb.grid(column=2, row=16,padx=15)


# User input for Di
Dim = ttk.Entry(window)
Dim.grid(row=13, column=3)
DiM = ttk.Entry(window)
DiM.grid(row=13, column=4)
comboDi = ttk.Combobox(window)
comboDi['values']= (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
comboDi.current(4) #set the selected item
comboDi.grid(column=5, row=13)
Dinum = comboDi.get()

# User input for Qi
Qim = ttk.Entry(window)
Qim.grid(row=14, column=3)
QiM = ttk.Entry(window)
QiM.grid(row=14, column=4)
comboQi = ttk.Combobox(window)
comboQi['values']= (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
comboQi.current(4) #set the selected item
comboQi.grid(column=5, row=14)
Qinum = comboQi.get()

# User input for Qt
Qtm = ttk.Entry(window)
Qtm.grid(row=15, column=3)
QtM = ttk.Entry(window)
QtM.grid(row=15, column=4)
comboQt = ttk.Combobox(window)
comboQt['values']= (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
comboQt.current(4) #set the selected item
comboQt.grid(column=5, row=15)
Qtnum = comboQt.get()

# User input for b
bm = ttk.Entry(window)
bm.grid(row=16, column=3)
bM = ttk.Entry(window)
bM.grid(row=16, column=4)
combob = ttk.Combobox(window)
combob['values']= (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
combob.current(4) #set the selected item
combob.grid(column=5, row=16)
bnum = combob.get()

# Button for generating production history
btn = ttk.Button(window, text="Production History", command=declinecurveanalysis) 
btn.grid(column=3, row=20, rowspan=2,pady=10)

# Button for generating the probability curves and data
btn = ttk.Button(window, text="RUN", command=RUN) 
btn.grid(column=5, row=20, rowspan=2,pady=10)

window.protocol("WM_DELETE_WINDOW", on_closing)
window.mainloop()


C:\Users\arham\anaconda3\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
C:\Users\arham\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\arham\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in power
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\arham\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-4fdf4d30c05b>", line 239, in RUN
    Maxb,Minb,numb,vol,NpDCA,T0,Q0)
  File "<ipython-input-1-4fdf4d30c05b>", line 90, in iterativetool
    maxQi,maxQt,maxDi,maxb = listing[int(Maxindex[0])]
TypeError: only size-1 arrays can be converted to Python scalars
C:\Users\arham\anaconda3\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Attempting to set id